# Live Webscraper for Current Price: get_current_stock_price(name)

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib3 as url
import certifi as cert

In [2]:
def get_current_stock_price(name):
    http = url.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=cert.where())
    html_doc = http.request('GET', 'https://finance.yahoo.com/quote/' + name + '?p=' + name)
    soup = BeautifulSoup(html_doc.data, 'html.parser')
    dayprice = soup.find("span", class_="Trsdu(0.3s) Fw(b) Fz(36px) Mb(-4px) D(ib)").get_text()
    prevclose = soup.find("td", class_="Ta(end) Fw(b) Lh(14px)").get_text()
    return 'dayprice', dayprice,  ' prevclose' , prevclose

In [3]:
get_current_stock_price('AAPL')

('dayprice', '166.97', ' prevclose', '167.96')

# Import Historical Data Using Quandl API: quandl_stocks(symbol)
# Stack Functions: compute_returns(quandl_stocks(symbol))

In [4]:
import quandl
import datetime
import time
t0 = time.time()

In [5]:
quandl.ApiConfig.api_key = '4ERUKenEWF5tsExbY9XP'

In [6]:
def quandl_stocks(symbol, start_date=(2000, 1, 1), end_date=None):
    """
    symbol is a string representing a stock symbol, e.g. 'AAPL'
 
    start_date and end_date are tuples of integers representing the year, month,
    and day
 
    end_date defaults to the current date when None
    """
 
    query_list = ['WIKI' + '/' + symbol + '.' + str(k) for k in range(1, 13)]
 
    start_date = datetime.date(*start_date)
 
    if end_date:
        end_date = datetime.date(*end_date)
    else:
        end_date = datetime.date.today()
 
    df =  quandl.get(query_list, 
            returns='pandas', 
            start_date=start_date,
            end_date=end_date,
            collapse='daily',
            order='asc'
            )
    return df

In [7]:
def compute_returns(df):
    daily_return = df.iloc[:,3].pct_change(1)
    daily_return.name = 'daily_return'
    monthly_return = df.iloc[:,3].pct_change(21)
    monthly_return.name = 'monthly_return'
    yearly_return = df.iloc[:,3].pct_change(252)
    yearly_return.name = 'yearly_return'
    returns = [df, daily_return, monthly_return,yearly_return]
    df_with_returns = pd.concat(returns, axis=1, join_axes=[df.index])
    return df_with_returns

In [8]:
aapl_returns = compute_returns(quandl_stocks('AAPL'))
aapl_returns

,WIKI/AAPL - Open,WIKI/AAPL - High,WIKI/AAPL - Low,WIKI/AAPL - Close,WIKI/AAPL - Volume,WIKI/AAPL - Ex-Dividend,WIKI/AAPL - Split Ratio,WIKI/AAPL - Adj. Open,WIKI/AAPL - Adj. High,WIKI/AAPL - Adj. Low,WIKI/AAPL - Adj. Close,WIKI/AAPL - Adj. Volume,daily_return,monthly_return,yearly_return
Date,,,,,,,,,,,,,,,
2000-01-03,104.870,112.5000,101.6900,111.94,4783900.0,0.0,1.0,3.369314,3.614454,3.267146,3.596463,133949200.0,NaN,NaN,NaN
2000-01-04,108.250,110.6200,101.1900,102.50,4574800.0,0.0,1.0,3.477908,3.554053,3.251081,3.293170,128094400.0,-0.084331,NaN,NaN
2000-01-05,103.750,110.5600,103.0000,104.00,6949300.0,0.0,1.0,3.333330,3.552125,3.309234,3.341362,194580400.0,0.014634,NaN,NaN
2000-01-06,106.120,107.0000,95.0000,95.00,6856900.0,0.0,1.0,3.409475,3.437748,3.052206,3.052206,191993200.0,-0.086538,NaN,NaN
2000-01-07,96.500,101.0000,95.5000,99.50,4113700.0,0.0,1.0,3.100399,3.244977,3.068270,3.196784,115183600.0,0.047368,NaN,NaN
2000-01-10,102.000,102.2500,94.7500,97.75,4509500.0,0.0,1.0,3.277105,3.285138,3.044174,3.140559,126266000.0,-0.017588,NaN,NaN
2000-01-11,95.940,99.3700,90.5000,92.75,3942400.0,0.0,1.0,3.082407,3.192607,2.907628,2.979917,110387200.0,-0.051151,NaN,NaN
2000-01-12,95.000,95.5000,86.5000,87.19,8714900.0,0.0,1.0,3.052206,3.068270,2.779114,2.801283,244017200.0,-0.059946,NaN,NaN
2000-01-13,94.480,98.7500,92.5000,96.75,9220400.0,0.0,1.0,3.035499,3.172688,2.971885,3.108431,258171200.0,0.109646,NaN,NaN


In [9]:
googl_returns = compute_returns(quandl_stocks('GOOGL'))
googl_returns

,WIKI/GOOGL - Open,WIKI/GOOGL - High,WIKI/GOOGL - Low,WIKI/GOOGL - Close,WIKI/GOOGL - Volume,WIKI/GOOGL - Ex-Dividend,WIKI/GOOGL - Split Ratio,WIKI/GOOGL - Adj. Open,WIKI/GOOGL - Adj. High,WIKI/GOOGL - Adj. Low,WIKI/GOOGL - Adj. Close,WIKI/GOOGL - Adj. Volume,daily_return,monthly_return,yearly_return
Date,,,,,,,,,,,,,,,
2004-08-19,100.010,104.06,95.96,100.335,44659000.0,0.0,1.0,50.159839,52.191109,48.128568,50.322842,44659000.0,NaN,NaN,NaN
2004-08-20,101.010,109.08,100.50,108.310,22834300.0,0.0,1.0,50.661387,54.708881,50.405597,54.322689,22834300.0,0.079484,NaN,NaN
2004-08-23,110.760,113.48,109.05,109.400,18256100.0,0.0,1.0,55.551482,56.915693,54.693835,54.869377,18256100.0,0.010064,NaN,NaN
2004-08-24,111.240,111.60,103.57,104.870,15247300.0,0.0,1.0,55.792225,55.972783,51.945350,52.597363,15247300.0,-0.041408,NaN,NaN
2004-08-25,104.760,108.00,103.88,106.000,9188600.0,0.0,1.0,52.542193,54.167209,52.100830,53.164113,9188600.0,0.010775,NaN,NaN
2004-08-26,104.950,107.95,104.66,107.910,7094800.0,0.0,1.0,52.637487,54.142132,52.492038,54.122070,7094800.0,0.018019,NaN,NaN
2004-08-27,108.100,108.62,105.69,106.150,6211700.0,0.0,1.0,54.217364,54.478169,53.008633,53.239345,6211700.0,-0.016310,NaN,NaN
2004-08-30,105.280,105.49,102.01,102.010,5196700.0,0.0,1.0,52.802998,52.908323,51.162935,51.162935,5196700.0,-0.039001,NaN,NaN
2004-08-31,102.320,103.71,102.16,102.370,4917800.0,0.0,1.0,51.318415,52.015567,51.238167,51.343492,4917800.0,0.003529,NaN,NaN


# Plot two stock prices on a bokeh plot 

In [10]:
from bokeh.plotting import figure, show
from bokeh.palettes import Spectral6
from bokeh.io import output_notebook

In [11]:
def plot_two_stock_price_common_date(df1,df2):
    earliest_comdate = max(df1.index[1],df2.index[1])
    latest_comdate = min(df1.index[len(df1.index)-1],df2.index[len(df2.index)-1])
    df1_reindex = df1[earliest_comdate <=df1.index]
    df1_reindex= df1_reindex[df1_reindex.index<= latest_comdate]
    df2_reindex = df2[earliest_comdate <=df2.index]
    df2_reindex= df2_reindex[df2_reindex.index<= latest_comdate]
    combined_close_price_df = pd.concat([df1_reindex.iloc[:,3], df2_reindex.iloc[:,3]],axis=1, join_axes=[df1_reindex.index])
    numlines=len(combined_close_price_df.columns)
    mypalette=Spectral6[0:numlines]
    output_notebook()
    p = figure(width=1000, height=600, x_axis_type="datetime") 
    p.line(combined_close_price_df.index.values, combined_close_price_df.iloc[:,0].values, legend = combined_close_price_df.columns.values[0], line_color=mypalette[0], line_width=2)
    p.line(combined_close_price_df.index.values, combined_close_price_df.iloc[:,1].values, legend = combined_close_price_df.columns.values[1], line_color=mypalette[1], line_width=2)
    show(p)

In [12]:
plot_two_stock_price_common_date(aapl_returns,googl_returns)
t1 = time.time()
total = t1-t0

Loading BokehJS ...

In [13]:
print ('total time taken =' + str(total) + 's')

total time taken =36.39696002006531s
